In [137]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import random
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv("dataset.csv").fillna("0").sort_values("Disease").reset_index(drop=True)
cat_col = df.select_dtypes("object").columns
df[cat_col] = df[cat_col].astype("category")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Disease     4920 non-null   category
 1   Symptom_1   4920 non-null   category
 2   Symptom_2   4920 non-null   category
 3   Symptom_3   4920 non-null   category
 4   Symptom_4   4920 non-null   category
 5   Symptom_5   4920 non-null   category
 6   Symptom_6   4920 non-null   category
 7   Symptom_7   4920 non-null   category
 8   Symptom_8   4920 non-null   category
 9   Symptom_9   4920 non-null   category
 10  Symptom_10  4920 non-null   category
 11  Symptom_11  4920 non-null   category
 12  Symptom_12  4920 non-null   category
 13  Symptom_13  4920 non-null   category
 14  Symptom_14  4920 non-null   category
 15  Symptom_15  4920 non-null   category
 16  Symptom_16  4920 non-null   category
 17  Symptom_17  4920 non-null   category
dtypes: category(18)
memory usage: 103.9 KB


In [3]:
df[cat_col].nunique().reset_index(name='cardinality')

,index,cardinality
0,Disease,41
1,Symptom_1,34
2,Symptom_2,48
3,Symptom_3,54
4,Symptom_4,51
5,Symptom_5,39
6,Symptom_6,33
7,Symptom_7,27
8,Symptom_8,22
9,Symptom_9,23


In [4]:
df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinning_movements,unsteadiness,0,0,0,0,0,0,0,0,0,0,0,0
1,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinning_movements,loss_of_balance,unsteadiness,0,0,0,0,0,0,0,0,0,0,0
2,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinning_movements,loss_of_balance,0,0,0,0,0,0,0,0,0,0,0,0
3,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinning_movements,loss_of_balance,unsteadiness,0,0,0,0,0,0,0,0,0,0,0
4,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinning_movements,loss_of_balance,unsteadiness,0,0,0,0,0,0,0,0,0,0,0


In [5]:
for col in cat_col:
    df[col] = df[col].str.replace(" ", "")

In [8]:
data_encoder = LabelEncoder()
sym = pd.read_csv("symptom_severity.csv").Symptom
sym = list(sym) + ["0"]
data_encoder = data_encoder.fit(sym)
print (len(data_encoder.classes_))
#np.save('data_classes.npy', data_encoder.classes_)

133


In [7]:
#df.to_csv("clean_dataset.csv", index=False)
encoder = LabelEncoder()
encoder = encoder.fit(df.Disease)
print (encoder.classes_)
#np.save('labels_classes.npy', encoder.classes_)

['(vertigo)ParoymsalPositionalVertigo' 'AIDS' 'Acne' 'Alcoholichepatitis'
 'Allergy' 'Arthritis' 'BronchialAsthma' 'Cervicalspondylosis'
 'Chickenpox' 'Chroniccholestasis' 'CommonCold' 'Dengue' 'Diabetes'
 'Dimorphichemmorhoids(piles)' 'DrugReaction' 'Fungalinfection' 'GERD'
 'Gastroenteritis' 'Heartattack' 'HepatitisB' 'HepatitisC' 'HepatitisD'
 'HepatitisE' 'Hypertension' 'Hyperthyroidism' 'Hypoglycemia'
 'Hypothyroidism' 'Impetigo' 'Jaundice' 'Malaria' 'Migraine'
 'Osteoarthristis' 'Paralysis(brainhemorrhage)' 'Pepticulcerdiseae'
 'Pneumonia' 'Psoriasis' 'Tuberculosis' 'Typhoid' 'Urinarytractinfection'
 'Varicoseveins' 'hepatitisA']


In [147]:
class model_embed:
    random.seed(10) # seeding for reproductibility
    
    def __init__(self, data, cv, encoder, encoder_data, name_y):
        self.name = "Keras Categorical Embeding"
        self.data = data
        self.cv = cv
        self.class_encoder = encoder
        self.y_name = name_y
        self.data_encoder = encoder_data
        self.model_ = self.model()
        
        
    def split(self):
        """split data into train, val, and test index"""
        cv_index = {}
        for index in range(0, self.cv):
            split_train = []
            split_val = []
            split_test = []
            train_test_split = 0
            for index_data in self.data.index:
                if index_data % 120 == 0 and index_data != 0:
                    for index_y in (random.sample(list(np.arange(index_data-120, index_data)), 24)):
                        split_test.append(int(index_y))
        
            #print ("done ", index+1)
            train_test_split = [train_idx for train_idx in (self.data.index) if train_idx not in split_test]
            split_val = list(np.random.choice(train_test_split, size=(18)))
            split_train = [train_idx for train_idx in train_test_split if train_idx not in split_val]
            cv_index[(index)] = np.array([(split_train), (split_val), list(split_test)])
        return cv_index
    def model(self):
        """creating the model"""
        model_data = tf.keras.Sequential([
            tf.keras.layers.Embedding(134, 328, input_length=17),
            tf.keras.layers.LSTM(32),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(164, activation="relu"),
            tf.keras.layers.Dense(82, activation="relu"),
            tf.keras.layers.Dense(41, activation="softmax")
        ])
        return model_data
        
    def model_train(self,data_x, data_y, val_x, val_y, test_x, test_y, num_epochs=5):
        """train model with specifid cross validation"""
        self.model_.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        history = self.model_.fit(data_x, data_y, epochs=num_epochs, batch_size=32, verbose=1, validation_data=(val_x, val_y))
        return self.model_.evaluate(test_x, test_y)
    
    def predict(self, test_x):
        data_p = self.data_encoder.transform(test_x)
        data_p = np.reshape(data_p, [1, 17])
        prediction = (self.model_.predict(data_p))
        label = self.class_encoder.inverse_transform([np.argmax(prediction)])
        return label
        
    def data_slicing(self, cv, x):
        """return data x and y for every cv"""
        train =  self.data.iloc[cv[0]]
        train_x = train.drop([self.y_name], axis=1)
        for col in train_x.columns:
            train_x[col] = self.data_encoder.transform(train_x[col].values)
        train_y = self.class_encoder.transform(train[self.y_name].values)
        
        val = self.data.iloc[cv[1]]
        val_x = val.drop([self.y_name], axis=1)
        for col in val_x.columns:
            val_x[col] = self.data_encoder.transform(val_x[col].values)
        val_y =  self.class_encoder.transform(val[self.y_name])
        
        test = self.data.iloc[cv[2]]
        test_x = test.drop([self.y_name], axis=1)
        for col in test_x.columns:
            test_x[col] = self.data_encoder.transform(test_x[col].values)
        test_y =  self.class_encoder.transform(test[self.y_name])
        return train_x, train_y, val_x, val_y, test_x, test_y
        
    def run(self):
        data_cv = self.split()
        result = []
        for x in tqdm(range(self.cv)):
            print (x)
            train_x, train_y, val_x, val_y, test_x, test_y = (self.data_slicing(data_cv[x], x))
            #print (self.model_)
            #print (train_x.shape, val_x.shape, test_x.shape)
            #print (train_y.shape, val_y.shape, test_y.shape)
            result.append(self.model_train(train_x, train_y, val_x, val_y, test_x, test_y, 5)[1])
        #print (self.class_encoder.classes_)
        print (sum(result)/self.cv)
        

In [148]:
m = model_embed(df, 1, encoder, data_encoder, "Disease")
m.run()

C:\Users\musth\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


  0%|          | 0/1 [00:00<?, ?it/s]

0
Epoch 1/5
124/124 [==============================] - 7s 13ms/step - loss: 3.3280 - accuracy: 0.1103 - val_loss: 1.1809 - val_accuracy: 0.6667
Epoch 2/5
124/124 [==============================] - 1s 10ms/step - loss: 0.7283 - accuracy: 0.8338 - val_loss: 0.2618 - val_accuracy: 0.8333
Epoch 3/5
124/124 [==============================] - 1s 9ms/step - loss: 0.1214 - accuracy: 0.9693 - val_loss: 0.1205 - val_accuracy: 0.9444
Epoch 4/5
124/124 [==============================] - 1s 9ms/step - loss: 0.0393 - accuracy: 0.9926 - val_loss: 0.0124 - val_accuracy: 1.0000
Epoch 5/5
30/30 [==============================] - 0s 4ms/step - loss: 0.0051 - accuracy: 1.0000
1.0


In [149]:
random_nmbr = random.sample(list(np.arange(0, 4920)), 1)
test_x = df.iloc[random_nmbr].drop(["Disease"], axis=1).values
test_y = df.iloc[random_nmbr]["Disease"].values
y = m.predict(test_x[0])
print (y.item(), test_y.item())

HepatitisC HepatitisC


In [150]:
m.model().summary()

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_37 (Embedding)     (None, 17, 328)           43952     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                46208     
_________________________________________________________________
flatten_31 (Flatten)         (None, 32)                0         
_________________________________________________________________
dense_108 (Dense)            (None, 164)               5412      
_________________________________________________________________
dense_109 (Dense)            (None, 82)                13530     
_________________________________________________________________
dense_110 (Dense)            (None, 41)                3403      
Total params: 112,505
Trainable params: 112,505
Non-trainable params: 0
_______________________________________________

In [151]:
model_learned = m.model()
model_learned.save('embedding_lstm')

INFO:tensorflow:Assets written to: embedding_lstm\assets


INFO:tensorflow:Assets written to: embedding_lstm\assets


In [5]:
# Model Load
from tensorflow import keras
model = keras.models.load_model('base_lstm')
# Encoder Load
data_enc = LabelEncoder()
data_enc.classes_ = np.load("data_classes.npy", allow_pickle=True)

label_enc = LabelEncoder()
label_enc.classes_ = np.load("labels_classes.npy", allow_pickle=True)

In [39]:
x = list(df[0:1].drop("Disease", axis=1).values)
y = df[0:1].Disease

In [42]:
x

[array(['vomiting', 'headache', 'nausea', 'spinning_movements',
        'unsteadiness', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        '0', '0'], dtype=object)]

In [41]:
x_mod = data_enc.transform(np.reshape(x, [17, 1]))
x_mod

array([123,  45,  72, 105, 121,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [30]:
x_mod_p = np.reshape(x_mod, [1, 1, 17])
x_mod_p

array([[[123,  45,  72, 105, 121,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0]]])

In [38]:
label_enc.inverse_transform([np.argmax(model.predict(x_mod_p))])[0]

'(vertigo)ParoymsalPositionalVertigo'